#### 2.1.11 _Programming Exercise:_ Generate segwit v1 addresses for a 2-of-2 MuSig aggregate pubkey

In [ ]:
# Generate a 2-of-2 aggregate MuSig key using the same pubkeys as before
# Method: generate_musig_key(ECPubKey_list)
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Multiply individual keys with challenges
privkey1_c = c_map[pubkey1] * privkey1
privkey2_c = c_map[pubkey2] * privkey2
pubkey1_c = c_map[pubkey1] * pubkey1
pubkey2_c = c_map[pubkey2] * pubkey2

if agg_pubkey.get_y()%2 != 0:
    privkey1_c.negate()
    privkey2_c.negate()
    pubkey1_c.negate()
    pubkey2_c.negate()
    agg_pubkey.negate()

# Create a segwit v1 address for the MuSig aggregate pubkey
# Method: address = program_to_witness(version_int, program_bytes)
program_musig = agg_pubkey.get_bytes()
address_musig = program_to_witness(0x01, program_musig)
print("2-of-2 musig: ", address_musig)

#### 2.1.14 _Programming Exercise:_ Create a valid BIP340 signature for the MuSig aggregate pubkey

In [ ]:
# Create sighash for ALL (0x00)
sighash_musig = TaprootSignatureHash(spending_tx, [tx.vout[0]], SIGHASH_ALL_TAPROOT, input_index=0)

# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces(nonce_list)
nonce1 = generate_schnorr_nonce()
nonce2 = generate_schnorr_nonce()
R_agg, negated = aggregate_schnorr_nonces([nonce1.get_pubkey(), nonce2.get_pubkey()])

if negated:
    nonce1.negate()
    nonce2.negate()

# Create an aggregate signature
# Method: sign_musig(privkey, nonce, R_agg, agg_pubkey, sighash_musig)
# Method: aggregate_musig_signatures(partial_signature_list, R_agg)
s1 = sign_musig(privkey1_c, nonce1, R_agg, agg_pubkey, sighash_musig)
s2 = sign_musig(privkey2_c, nonce2, R_agg, agg_pubkey, sighash_musig)
sig_agg = aggregate_musig_signatures([s1, s2], R_agg)
print("Aggregate signature is {}\n".format(sig_agg.hex()))

# Add witness to transaction
spending_tx.wit.vtxinwit.append(CTxInWitness([sig_agg]))

# Get transaction weight
print("Transaction weight: {}\n".format(node.decoderawtransaction(spending_tx.serialize().hex())['weight']))

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")